# Análisis Exploratorio de Datos - Diabetes Gestacional (GDM)

**Universidad de Magallanes**  
**Facultad de Ingeniería, Departamento de Ingeniería en Computación**  
**Asignatura:** Matemática para Ciencias de la Computación  
**Profesor:** David Medina Ortiz  

---

**Estudiante(s):** [Nombre(s) del/los estudiante(s)]  
**Fecha de entrega:** 15 de noviembre de 2025  

---

## Tabla de Progreso Global

| Sección | Título | Progreso | Estado |
|---------|--------|----------|--------|
| 1 | Descripción General del Dataset | 100% | ✅ |
| 2.1 | Análisis Exploratorio de Datos (EDA) | 56% | 🟡 |
| 2.2 | Intervalos de Confianza | 54% | 🟡 |
| 2.3 | Pruebas de Hipótesis | 8% | 🔴 |
| 2.4 | Evaluación de Normalidad | 85% | 🟢 |
| 2.5 | Análisis Bivariado | 28% | 🔴 |
| 2.6 | Interpretación Clínica y Conclusiones | 0% | 🔴 |
| 3 | Entregables y Archivos | 100% | ✅ |
| **GLOBAL** | **Progreso Total** | **39%** | 🟡 |

---

# 1. Descripción General del Dataset

## Contexto

Este trabajo utiliza un **dataset sintético** que simula información clínica del **primer trimestre del embarazo** para el estudio de la **diabetes gestacional (GDM)**. El objetivo es caracterizar estadísticamente los datos y explorar asociaciones entre variables clínicas y el riesgo de desarrollar GDM.

## Características del Dataset

- **N**: Aproximadamente 1500 registros de pacientes embarazadas
- **Variables incluidas**:
  - **Clínicas**: edad, IMC pregestacional, presión arterial (sistólica, diastólica, MAP), semanas de gestación
  - **Bioquímicas**: FPG (glucosa en ayunas), HbA1c, insulina, HOMA-IR, triglicéridos, HDL
  - **Factores de riesgo**: paridad, antecedentes familiares de diabetes tipo 2, GDM previa, PCOS, tabaquismo
  - **Estilo de vida**: nivel de actividad física, puntuación de dieta (0-100)
  - **Variable objetivo**: `label_gdm` (0 = No GDM, 1 = GDM)

## Características Especiales

- **Datos faltantes**: Presencia de valores nulos con patrones MCAR (Missing Completely At Random) y MAR (Missing At Random)
- **Outliers**: El dataset contiene valores atípicos que requieren identificación y tratamiento
- **Desbalance de clases**: Aproximadamente 17% de casos positivos de GDM
- **Tipos de variables**: Mixto (continuas, discretas y categóricas binarias)

## Objetivo del Análisis

Realizar un análisis exploratorio exhaustivo que permita:
1. Caracterizar la distribución de las variables
2. Identificar patrones y asociaciones relevantes
3. Comparar grupos (GDM vs No-GDM)
4. Evaluar supuestos estadísticos
5. Generar insights clínicos para la comprensión del riesgo de GDM

---

# 2. Actividades Desarrolladas

Esta sección contiene el análisis exploratorio organizado por subsecciones según la guía práctica.

---

## 2.1 Análisis Exploratorio de Datos (EDA)

**Estado**: 🟡 56% completado  
**Pendiente**: Visualizaciones adicionales e interpretación textual de variables

### Tareas realizadas:
- ✅ Carga de datos y verificación de dimensiones
- ✅ Análisis de tipos de datos
- ✅ Detección de valores faltantes
- ✅ Estadística descriptiva (media, mediana, IQR, percentiles)
- ✅ Detección de outliers por IQR e Isolation Forest
- ✅ Filtrado de outliers por votación
- ✅ Visualizaciones iniciales (histogramas, boxplots, violinplots)

### Tareas pendientes:
- ⏳ Histogramas individuales con interpretación
- ⏳ Superposición de curva normal en histogramas
- ⏳ Gráficos de densidad (KDE) para variables clave
- ⏳ Interpretación textual detallada por variable
- ⏳ Documentación de estrategia para outliers

---

### Importación de librerías

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from auxiliar_functions import *
from sklearn.ensemble import IsolationForest
from scipy import stats

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

### Carga y exploración inicial del dataset

In [ ]:
# Carga del dataset
df_data = pd.read_csv("gdm_first_trimester_ml_dataset.csv")

# Primeras filas
print("Primeras 5 filas del dataset:")
df_data.head(5)

In [ ]:
# Dimensiones del dataset
print(f"Dimensiones: {df_data.shape}")
print(f"Total de registros: {df_data.shape[0]}")
print(f"Total de variables: {df_data.shape[1]}")
df_data.shape

In [ ]:
# Distribución de la variable objetivo
print("Distribución de GDM:")
print(df_data["label_gdm"].value_counts())
print(f"\nProporción de casos positivos: {df_data['label_gdm'].mean():.2%}")

In [ ]:
# Tipos de datos
print("Tipos de datos por variable:")
df_data.dtypes

### Análisis de valores faltantes

In [ ]:
df_nulls = df_data.isna().astype(int)
df_summary_null = generate_df_counts(df_nulls, columns_name=["descriptor", "count_Nulls", "count_Falses"])
print("Resumen de valores faltantes:")
df_summary_null

### Estadística descriptiva

Definimos las variables categóricas que se excluirán del análisis descriptivo de variables continuas:

In [ ]:
columns_to_ignore = [
    "parity", 
    "family_history_t2d",
    "previous_gdm",
    "pcos", 
    "smoking_first_trimester",
    "label_gdm",
    "physical_activity_level"
]

print("Variables categóricas/binarias a excluir del análisis descriptivo:")
print(columns_to_ignore)

In [ ]:
# Estadística descriptiva para todas las variables
print("Estadísticas descriptivas del dataset completo:")
df_data.describe()

In [ ]:
statistical_descriptors = []

for column in df_data.columns:
    if column not in columns_to_ignore:
        descriptive_values = df_data[column].describe()

        IQR = descriptive_values["75%"] - descriptive_values["25%"]
        min_value, max_value = get_range_outlier(
            descriptive_values["25%"], 
            descriptive_values["75%"], 
            IQR)

        row = {
            "descriptor" : column,
            "mean":descriptive_values["mean"],
            "std" :descriptive_values["std"],
            "median" : descriptive_values["50%"],
            "IQR" : IQR,
            "25%" : descriptive_values["25%"],
            "75%" :descriptive_values["75%"],
            "min_value_for_outlier" : min_value,
            "max_value_for_outlier" : max_value

        }
        statistical_descriptors.append(row)

df_statistical = pd.DataFrame(statistical_descriptors)
print("Estadísticas descriptivas con IQR y umbrales para outliers:")
df_statistical

### Detección de outliers

**Nota**: Se implementa detección por IQR e Isolation Forest con sistema de votación.

---

## 2.2 Intervalos de Confianza

**Estado**: 🟡 54% completado  
**Pendiente**: IC para proporciones, diferencias de medias e interpretación clínica

### Tareas realizadas:
- ✅ Funciones de cálculo de IC implementadas en `auxiliar_functions.py`
- ✅ IC para media y varianza de IMC, FPG, HbA1c con múltiples tamaños de muestra

### Tareas pendientes:
- ⏳ IC para medias de: HDL, presión arterial (sistólica, diastólica, MAP), triglicéridos
- ⏳ IC para proporciones (fumadoras, antecedentes familiares, PCOS, GDM previa)
- ⏳ IC para diferencia de medias entre grupos GDM vs No-GDM
- ⏳ Interpretación clínica comparando con rangos de referencia

---

### Cálculo de intervalos de confianza para diferentes tamaños de muestra

In [ ]:
n = np.linspace(10, 100, 10)  # Tamaños de muestra de 10 a 100
print("Tamaños de muestra a evaluar:")
print(n)

nro_tamanhos = n.size
np.random.seed(42)

# Intervalos de confianza
for i in range(0, nro_tamanhos):
    df_mini = pd.DataFrame()
    picked_up_indexs = []
    index = None

    for j in range(0, int(n[i])):    
        while index in picked_up_indexs:
            index = np.random.randint(0, df_data.shape[0])
            df_mini = pd.concat([df_mini, df_data.iloc[[index]]])
        picked_up_indexs.append(index)
    
    # Intervalos de confianza para variables clave
    interesting_descriptors_names = ["bmi_prepreg_kg_m2", "fpg_mmol_l", "hba1c_percent"]

    for descriptor_name in interesting_descriptors_names:
        descriptor = df_mini[descriptor_name]
        descriptor_mean = descriptor.mean()
        descriptor_std = descriptor.std()
        IC_mean = calculate_ic_mean(descriptor_mean, descriptor_std, int(n[i]))
        IC_std = calculate_ic_std(descriptor_std, int(n[i]))
        print(f"Para n={int(n[i])} el intervalo de confianza para la media (μ) del descriptor '{descriptor_name}' es [{IC_mean[0]:.4f}, {IC_mean[1]:.4f}]")
        print(f"Para n={int(n[i])} el intervalo de confianza para la varianza (σ²) del descriptor '{descriptor_name}' es [{IC_std[0]:.4f}, {IC_std[1]:.4f}]")
    
    print()

---

## 2.3 Pruebas de Hipótesis

**Estado**: 🔴 8% completado - **CRÍTICO**  
**Pendiente**: Casi todas las pruebas de hipótesis

### Tareas realizadas:
- ✅ Función `compare_two_groups_numeric` implementada (t-test/Welch/Mann-Whitney automático)
- ✅ Comparación preliminar de presión arterial (sistólica, diastólica, MAP) entre grupos GDM

### Tareas pendientes:
- ⏳ Formular H₀ y H₁ para todas las comparaciones
- ⏳ Comparar IMC, FPG, HbA1c entre GDM vs No-GDM
- ⏳ Comparar dieta según nivel de actividad física (ANOVA/Kruskal-Wallis)
- ⏳ Pruebas de proporciones (chi² o z): antecedentes familiares, PCOS, tabaquismo vs GDM
- ⏳ Reportar tamaño de efecto (d de Cohen o r) para todas las pruebas
- ⏳ Interpretar resultados en contexto clínico

---

### Comparación de presión arterial entre grupos GDM (ejemplo preliminar)

In [ ]:
def compare_two_groups_numeric(df, y_col, group_col="label_gdm", alpha=0.05, tail="two-sided"):
    """
    Compara una variable numérica (y_col) entre dos grupos binarios (group_col).
    Verifica supuestos: normalidad por grupo (Shapiro) y homogeneidad (Levene).
    Decide automáticamente entre:
      - t de Student clásico (varianzas iguales)
      - Welch t-test (varianzas desiguales)
      - Mann–Whitney U (si falla normalidad)
    Retorna un dict con todo lo necesario para el informe.
    """
    d = df[[y_col, group_col]].dropna()
    g0 = d[d[group_col] == 0][y_col].astype(float).values
    g1 = d[d[group_col] == 1][y_col].astype(float).values

    # Hipótesis
    H0 = f"μ_{y_col}|GDM=0 == μ_{y_col}|GDM=1"
    H1 = f"μ_{y_col}|GDM=0 != μ_{y_col}|GDM=1" if tail=="two-sided" else f"μ diferencias unilaterales ({tail})"

    # Normalidad por grupo (solo si n>=3)
    sh0 = stats.shapiro(g0) if len(g0) >= 3 else (np.nan, np.nan)
    sh1 = stats.shapiro(g1) if len(g1) >= 3 else (np.nan, np.nan)
    normal_ok = (not np.isnan(sh0[1]) and sh0[1] > alpha) and (not np.isnan(sh1[1]) and sh1[1] > alpha)

    # Homogeneidad de varianzas (Levene)
    lev = stats.levene(g0, g1, center='median')
    equal_var = lev.pvalue > alpha

    # Elección de prueba
    if normal_ok:
        res = stats.ttest_ind(g0, g1, equal_var=equal_var, alternative="two-sided")
        test_name = "t-test (Welch)" if not equal_var else "t-test (varianzas iguales)"
        stat, p = res.statistic, res.pvalue
    else:
        res = stats.mannwhitneyu(g0, g1, alternative="two-sided")
        test_name = "Mann–Whitney U"
        stat, p = res.statistic, res.pvalue

    decision = "Rechazo H0" if p < alpha else "No rechazo H0"

    return {
        "variable": y_col,
        "n_gdm0": len(g0), "mean_gdm0": np.mean(g0), "sd_gdm0": np.std(g0, ddof=1),
        "n_gdm1": len(g1), "mean_gdm1": np.mean(g1), "sd_gdm1": np.std(g1, ddof=1),
        "shapiro_g0_p": (np.nan if np.isnan(sh0[1]) else sh0[1]),
        "shapiro_g1_p": (np.nan if np.isnan(sh1[1]) else sh1[1]),
        "levene_p": lev.pvalue,
        "test": test_name, "stat": stat, "p_value": p,
        "alpha": alpha, "decision": decision,
        "H0": H0, "H1": H1
    }

# Ejemplo: Comparación de presión arterial
print("Comparación de presión arterial entre grupos GDM vs No-GDM:")
print("=" * 70)
for col in ["systolic_bp_mmHg", "diastolic_bp_mmHg", "map_mmHg"]:
    out = compare_two_groups_numeric(df_data, y_col=col, group_col="label_gdm", alpha=0.05)
    print(f"\n{col}:")
    print(out)

---

## 2.4 Evaluación de Normalidad

**Estado**: 🟢 85% completado - **CASI COMPLETO**  
**Pendiente**: Solo visualizaciones (QQ-plots) y transformaciones

### Tareas realizadas:
- ✅ Prueba de Shapiro-Wilk para todas las variables continuas
- ✅ Prueba de Kolmogorov-Smirnov (Lilliefors) para todas las variables continuas
- ✅ Resumen tabular de resultados con decisión (Normal / No normal)
- ✅ Comparación de ambos tests y priorización de Shapiro-Wilk

### Tareas pendientes:
- ⏳ Generar QQ-plots para cada variable continua
- ⏳ Histogramas con curva normal superpuesta
- ⏳ Probar transformaciones (log, Box-Cox) en variables asimétricas
- ⏳ Re-evaluar normalidad post-transformación

---

### Pruebas de normalidad: Shapiro-Wilk y Kolmogorov-Smirnov

In [ ]:
continuous_variables = [
    "age_years", "bmi_prepreg_kg_m2", "systolic_bp_mmHg", "diastolic_bp_mmHg", 
    "map_mmHg", "gestational_weeks", "fpg_mmol_l", "hba1c_percent", 
    "insulin_uIU_ml", "homa_ir", "triglycerides_mmol_l", "hdl_mmol_l", 
    "physical_activity_level", "diet_score_0_100"
]

# Inicializar estructuras
clean_data_descriptors = []
n_val = []
n_invalid = []

# Limpiar NaN por variable
for var in continuous_variables:
    valid_data = df_data[var].dropna()
    n_validos = valid_data.size
    n_faltantes = df_data[var].isna().sum()

    clean_data_descriptors.append(valid_data)
    n_val.append(n_validos)
    n_invalid.append(n_faltantes)

# Prueba de normalidad
normality_results = []
alpha = 0.05

# Evaluar cada variable
for idx, var in enumerate(continuous_variables):
    x = clean_data_descriptors[idx]

    # Shapiro–Wilk
    W, p_sw = stats.shapiro(x)
    
    # KS-Lilliefors (usando parámetros estimados)
    mu, sigma = x.mean(), x.std(ddof=1)
    D, p_ks = stats.kstest(x, 'norm', args=(mu, sigma))
    
    # Decisión
    decision_sw = "Normal" if p_sw >= alpha else "No normal"
    decision_ks = "Normal" if p_ks >= alpha else "No normal"
    
    # Guardar resultado
    normality_results.append({
        "Variable": var,
        "n": len(x),
        "SW_W": W,
        "SW_p": p_sw,
        "SW_decisión": decision_sw,
        "KS_D": D,
        "KS_p": p_ks,
        "KS_decisión": decision_ks
    })

# Mostrar resultados
print("Resultados de pruebas de normalidad:")
print("=" * 100)
for result in normality_results:
    print(f"\n{result['Variable']}:")
    print(f"  n={result['n']}")
    print(f"  Shapiro-Wilk: W={result['SW_W']:.4f}, p={result['SW_p']:.4f} → {result['SW_decisión']}")
    print(f"  KS-Lilliefors: D={result['KS_D']:.4f}, p={result['KS_p']:.4f} → {result['KS_decisión']}")

---

## 2.5 Análisis Bivariado

**Estado**: 🔴 28% completado - **CRÍTICO**  
**Pendiente**: Visualizaciones completas e interpretación clínica

### Tareas realizadas:
- ✅ Matriz de correlación de Pearson calculada
- ✅ Heatmap de correlación generado
- ✅ Pairplot preliminar con visualización por grupo GDM

### Tareas pendientes:
- ⏳ Identificar y discutir correlaciones fuertes (|r| > 0.7)
- ⏳ Scatterplots con regresión para pares clave (FPG vs HbA1c, Insulina vs HOMA-IR, etc.)
- ⏳ Incluir ecuación de regresión y R² en gráficos
- ⏳ Boxplots comparativos por grupo GDM para variables clave
- ⏳ Interpretación clínica de asociaciones encontradas
- ⏳ Relacionar hallazgos con literatura sobre GDM

---

### Matriz de correlación y heatmap

In [ ]:
# Nota: Esta celda usa df_filter del prototipo. Asegurar que esté creado previamente con:
# df_filter = df_data[df_data["vote_outlier"]<3].drop(columns=['is_outlier_by_IQR', etc.])

# Calcular correlación de Pearson
df_corr_pearson = df_filter.drop(columns=["label_gdm"]).corr(method="pearson")

# Visualizar heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(data=df_corr_pearson, annot=True, fmt=".2f", cmap="Blues", cbar_kws={'label': 'Correlación de Pearson'})
plt.title("Matriz de Correlación de Pearson - Variables Continuas", fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nCorrelaciones fuertes encontradas (|r| > 0.7):")
print("=" * 50)
# Identificar correlaciones fuertes
for i in range(len(df_corr_pearson.columns)):
    for j in range(i+1, len(df_corr_pearson.columns)):
        corr_value = df_corr_pearson.iloc[i, j]
        if abs(corr_value) > 0.7:
            print(f"{df_corr_pearson.columns[i]} vs {df_corr_pearson.columns[j]}: r = {corr_value:.3f}")

---

## 2.6 Interpretación Clínica y Conclusiones

**Estado**: 🔴 0% completado - **CRÍTICO**  
**Pendiente**: Toda la sección de conclusiones

### Tareas pendientes:
- ⏳ Síntesis de variables con distribución no normal y consecuencias
- ⏳ Resumen de diferencias significativas entre grupos (GDM vs No-GDM)
- ⏳ Destacar dirección de diferencias (qué grupo tiene valores más altos/bajos)
- ⏳ Listar correlaciones clínicamente relevantes y su significado
- ⏳ Discutir posibles relaciones causales vs correlación espuria
- ⏳ Comentar sobre datos faltantes y su posible impacto
- ⏳ Discutir sesgos potenciales en la muestra
- ⏳ Mencionar variables confusoras no controladas
- ⏳ Proponer análisis multivariado (regresión logística)
- ⏳ Sugerir validación cruzada de modelos predictivos
- ⏳ Recomendar estudios adicionales

---

### Sección a completar

**Estructura sugerida para las conclusiones:**

1. **Resumen de hallazgos principales**
   - Variables no normales y sus implicancias
   - Diferencias significativas entre grupos
   - Correlaciones relevantes encontradas

2. **Interpretación clínica**
   - Significado de las diferencias entre GDM vs No-GDM
   - Relevancia de las correlaciones para el diagnóstico/pronóstico
   - Comparación con rangos clínicos de referencia

3. **Limitaciones del estudio**
   - Impacto de datos faltantes
   - Sesgos y variables confusoras
   - Naturaleza del dataset sintético

4. **Próximos pasos**
   - Análisis multivariado recomendado
   - Validación de modelos predictivos
   - Estudios complementarios necesarios

---

# 3. Entregables y Archivos Adjuntos

## Archivos incluidos en la entrega

Este trabajo práctico incluye los siguientes archivos que deben ser entregados en un archivo comprimido (`.tar.gz` o `.zip`):

### Archivos principales

1. **`Entregable.ipynb`** (este documento)
   - Notebook principal con todo el análisis organizado por secciones
   - Incluye código, visualizaciones e interpretaciones
   - Indicadores de progreso por sección

2. **`gdm_first_trimester_ml_dataset.csv`**
   - Dataset original con los datos clínicos
   - N ≈ 1500 registros de pacientes embarazadas
   - Variables clínicas, bioquímicas y de estilo de vida

3. **`gdm_first_trimester_ml_dataset_metadata.json`**
   - Metadata del dataset
   - Descripción de variables y tipos de datos
   - Información sobre valores faltantes y rangos esperados

4. **`auxiliar_functions.py`**
   - Funciones auxiliares implementadas para el análisis
   - Incluye funciones para:
     - Cálculo de intervalos de confianza (media, varianza, desviación)
     - Detección de outliers
     - Generación de resúmenes de conteos
     - Otras utilidades

### Archivos de referencia y documentación

5. **`INSTRUCCIONES.md`**
   - Guía práctica original del profesor
   - Descripción de actividades y entregables
   - Criterios de evaluación

6. **`TASKS.md` / `TASK2.md`**
   - Lista detallada de tareas pendientes
   - Organización por prioridad (`TASKS.md`) y por sección (`TASK2.md`)
   - Indicadores de progreso y estimaciones de tiempo

7. **`README.md` (en carpeta `__pycache__/`)**
   - Evaluación de estado del progreso
   - Retroalimentación detallada por sección
   - Porcentajes de avance y corrección

8. **`.gitignore`**
   - Configuración de archivos ignorados por Git
   - Incluye: `__pycache__/`, `*.pyc`, entornos virtuales, etc.

### Archivos de trabajo (prototipos)

9. **`1_check_data.ipynb`**
   - Prototipo inicial del análisis
   - Contiene desarrollo del código que luego se organizó en `Entregable.ipynb`

10. **`check_data.ipynb`**
    - Versión preliminar del prototipo (obsoleta)

---

## Herramientas utilizadas

### Software y entorno

- **Python 3.x**
- **Jupyter Notebook / JupyterLab**
- **Git** (control de versiones)

### Librerías principales

- **pandas**: Manipulación y análisis de datos
- **numpy**: Operaciones numéricas y álgebra lineal
- **matplotlib**: Visualización básica
- **seaborn**: Visualización estadística avanzada
- **scipy.stats**: Pruebas estadísticas (Shapiro-Wilk, KS, t-test, Mann-Whitney, Levene, etc.)
- **sklearn.ensemble.IsolationForest**: Detección de anomalías/outliers

### Herramientas de IA (si aplica)

**[Indicar aquí si se usó ChatGPT, GitHub Copilot u otra herramienta de IA, especificando para qué tareas]**

Ejemplo:
> Se utilizó GitHub Copilot para:
> - Generación de código auxiliar para funciones estadísticas
> - Sugerencias de visualización con seaborn
> - Redacción de comentarios y documentación

---

## Instrucciones de ejecución

### Requisitos previos

1. Instalar Python 3.8 o superior
2. Instalar las librerías necesarias:

```bash
pip install pandas numpy matplotlib seaborn scipy scikit-learn
```

### Ejecución del notebook

1. Abrir `Entregable.ipynb` en Jupyter Notebook/Lab
2. Asegurar que `gdm_first_trimester_ml_dataset.csv` y `auxiliar_functions.py` están en el mismo directorio
3. Ejecutar las celdas secuencialmente desde el inicio

### Notas importantes

- Algunas celdas en secciones incompletas (🔴) pueden fallar si no se han ejecutado celdas previas necesarias
- El dataset filtrado `df_filter` se genera en la sección 2.1 y se usa en secciones posteriores
- Los resultados de normalidad (sección 2.4) deben usarse para justificar elecciones de tests en sección 2.3

---

## Información de contacto

**Estudiante(s):** [Completar nombre(s)]  
**Correo:** [Completar correo(s)]  
**Fecha de entrega:** 15 de noviembre de 2025  
**Profesor:** David Medina Ortiz  
**Correo del profesor:** david.medina@umag.cl

---

**Fin del documento**